In [1]:
print("Started")
import numpy as np, pandas as pd
print("Pandas version - ", pd.__version__)
from pandas import MultiIndex, Int16Dtype

try:
    import seaborn as sns
except:
    !pip install seaborn
    import seaborn as sns
import matplotlib.pyplot as plt
#import cupy, cudf
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Importing RFE and LinearRegression
try:
    import statsmodels.api as sm
except:
    !python -m pip install statsmodels
    import statsmodels.api as sm

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model,tree,svm

try:
    import pickle
except:
    !pip install pickle5
    import pickle

from sklearn.metrics import mean_absolute_error
print("All necessary libraries imported")

Started
Pandas version -  1.4.3


C:\Users\Admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


All necessary libraries imported


In [2]:
traindf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

print(traindf.shape, " - Training dataset shape")
print(testdf.shape, " - Testing dataset shape")
print(sample_sub.shape, " - Sample Submission dataset shape")

(6036000, 8)  - Training dataset shape
(4024000, 7)  - Testing dataset shape
(4024000, 2)  - Sample Submission dataset shape


In [3]:
traindf.columns

# Adding feature

traindf['time_delta'] = traindf.groupby('breath_id')['time_step'].diff()
#testdf['time_delta'] = testdf.groupby('breath_id')['time_step'].diff()

In [4]:
common_features = ['breath_id','R','C','time_step','u_in','u_out']
numerical_features = ['time_step','u_in']
categorical_features = ['R','C','u_out','breath_id']

In [5]:
"""
#!Tensor Flow Addons

!pip install -U tensorflow-addons
!pip install -q "tqdm>=4.36.1"
print("Done 1")

try:
    import tensorflow as tf
except:
    !pip install tensorflow
    import tensorflow as tf

import tensorflow_addons as tfa
print("Done 2")

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
print("Done 1")

import tqdm

# quietly deep-reload tqdm
import sys
from IPython.lib import deepreload 

print("Tensor flow addons added")
"""

'\n#!Tensor Flow Addons\n\n!pip install -U tensorflow-addons\n!pip install -q "tqdm>=4.36.1"\nprint("Done 1")\n\ntry:\n    import tensorflow as tf\nexcept:\n    !pip install tensorflow\n    import tensorflow as tf\n\nimport tensorflow_addons as tfa\nprint("Done 2")\n\nfrom tensorflow.keras.datasets import mnist\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Dropout, Flatten\nprint("Done 1")\n\nimport tqdm\n\n# quietly deep-reload tqdm\nimport sys\nfrom IPython.lib import deepreload \n\nprint("Tensor flow addons added")\n'

In [6]:
"""
stdout = sys.stdout
sys.stdout = open('junk','w')
deepreload.reload(tqdm)
sys.stdout = stdout


tqdm.__version__
"""

"\nstdout = sys.stdout\nsys.stdout = open('junk','w')\ndeepreload.reload(tqdm)\nsys.stdout = stdout\n\n\ntqdm.__version__\n"

In [7]:
# Adding features in the dataframe

def add_features(df):
    #https://www.kaggle.com/code/papernist/google-brain
    df = df.copy()
    
    df_group = df.groupby(['breath_id'])
    
    
    feature_list = ['u_in', 'time_step', 'cross']
    
    df['cross']= df['u_in'] * df['u_out']
    df['area_out']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    
    # add
    df['air_flow_rate'] = df['u_out'] - (df['u_in']/100.0)
    df['air_flow_area'] = df['air_flow_rate'] * df['time_step']
    print("Step-1...Completed")
    
    # cumsum cummean
    df['one'] = 1
    df['count'] = df_group['one'].cumsum()
    for feature in feature_list:
        df[f'{feature}_cumsum'] = df_group[feature].cumsum()
        df[f'{feature}_cummean'] = df[f'{feature}_cumsum'] / df['count']
        
    print("Step-2 cumsum cummean ...Completed")
    # lagging
    use_lags = 4
    for lag in range(1, use_lags+1):
        for feature in feature_list:
            # lag 
            df[f'{feature}_lag_{lag}'] = df_group[feature].shift(lag)
            # inverse lag
            df[f'{feature}_lag_inverse_{lag}'] = df_group[feature].shift(-lag)

            # dif lag
            df[f'{feature}_lag_diff_{lag}'] = df[feature] - df[f'{feature}_lag_{lag}']

            # dif inverse lag
            df[f'{feature}_lag_inverse_diff_{lag}'] = df[feature] - df[f'{feature}_lag_inverse_{lag}']

            df = df.drop(columns=[f'{feature}_lag_{lag}', f'{feature}_lag_inverse_{lag}'])
        
    df = df.fillna(0)
    print("Step-3 lagging ...Completed")
    
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    df = df.fillna(0)
    print("Step-4 categorical ...Completed")
    
    rolling_list = [15]
    for roll in rolling_list:
        for feature in feature_list:
            df[[f'{roll}_{feature}_sum',f'{roll}_{feature}_min',
                f'{roll}_{feature}_max',f'{roll}_{feature}_mean']] = (df_group[feature]\
                                                                  .rolling(window=roll,min_periods=1)\
                                                                  .agg({f'{roll}_{feature}_sum':'sum',
                                                                        f'{roll}_{feature}_min':'min',
                                                                        f'{roll}_{feature}_max':'max',
                                                                        f'{roll}_{feature}_mean':'mean'})\
                                                                   .reset_index(level=0,drop=True))
    
    print("Step-5 Sliding window...Completed")
    print()
    
    df = df.fillna(0)
    df = df.drop(['id', 'breath_id','one','count'], axis=1)
    
    return df.astype(np.float16)

In [8]:
%%time
train = add_features(traindf)
#test = add_features(testdf)

print("*"*70)
print(train.shape, " - Feature added training dataset shape")
#print(test.shape, " - Features added testing dataset shape")

Step-1...Completed
Step-2 cumsum cummean ...Completed
Step-3 lagging ...Completed
Step-4 categorical ...Completed
Step-5 Sliding window...Completed

**********************************************************************
(6036000, 67)  - Feature added training dataset shape
Wall time: 5min 36s


In [9]:
binary_col = []
num_col = []

for columnname in train.columns:
  if len(set(train[columnname])) != 2:
    num_col.append(columnname)
  else:
    binary_col.append(columnname)

train_numcol = num_col.copy()
train_numcol.remove('pressure')

print(len(binary_col), "Number of Binary Categorical Columns")
print(len(num_col),"Number of Numerical Columns")
print(len(binary_col)+len(num_col), "Sum of Binary & Numerical Columns")

16 Number of Binary Categorical Columns
51 Number of Numerical Columns
67 Sum of Binary & Numerical Columns


In [10]:
df_train, df_test = train_test_split(train, train_size = 0.7, test_size = 0.3, random_state = 100)
print("Split Done")

from sklearn import metrics
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split

trainsplitcopy = df_train.copy()
testsplitcopy = df_test.copy()
traincopy = train.copy()

y_train = trainsplitcopy.pop('pressure')
X_train = trainsplitcopy

y_test = testsplitcopy.pop('pressure')
X_test = testsplitcopy

y_train_nosplit = traincopy.pop('pressure')
X_train_nosplit = traincopy



print("*"*70)
print("Splitting the data into X train & Y train")
print(X_train.shape, "X train shape")
print(y_train.shape, "Y train shape")

print("*"*70)
print("Splitting the data into X test & Y test")
print(X_test.shape, "X test shape")
print(y_test.shape, "Y test shape")

print("*"*70)
print("No Split Data")
print(X_train_nosplit.shape, "X train shape")
print(y_train_nosplit.shape, "Y train shape")


Split Done
**********************************************************************
Splitting the data into X train & Y train
(4225200, 66) X train shape
(4225200,) Y train shape
**********************************************************************
Splitting the data into X test & Y test
(1810800, 66) X test shape
(1810800,) Y test shape
**********************************************************************
No Split Data
(6036000, 66) X train shape
(6036000,) Y train shape


In [11]:
#train2 = train.copy()

In [12]:
#Scaling the features

scaler = MinMaxScaler().fit(X_train_nosplit[train_numcol])
train_scaled_nosplit = pd.DataFrame(scaler.transform(X_train_nosplit[train_numcol]),
                              columns = X_train_nosplit[train_numcol].columns)
train_scaled_nosplit.head()

,time_step,u_in,time_delta,cross,area_out,area,air_flow_rate,air_flow_area,u_in_cumsum,u_in_cummean,...,15_u_in_max,15_u_in_mean,15_time_step_sum,15_time_step_min,15_time_step_max,15_time_step_mean,15_cross_sum,15_cross_min,15_cross_max,15_cross_mean
0,0.000000,0.000834,0.000000,0.0,0.0,0.000000,0.499512,0.254395,0.000031,0.000834,...,0.000834,0.000834,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.011459,0.183960,0.134155,0.0,0.0,0.005760,0.408203,0.252686,0.006783,0.092346,...,0.183960,0.092346,0.000836,0.0,0.011459,0.006271,0.0,0.0,0.0,0.0
2,0.022980,0.225220,0.135010,0.0,0.0,0.014145,0.387451,0.250244,0.015045,0.136597,...,0.225220,0.136597,0.002514,0.0,0.022980,0.012566,0.0,0.0,0.0,0.0
3,0.034576,0.228149,0.135620,0.0,0.0,0.021561,0.385986,0.248169,0.023422,0.159546,...,0.228149,0.159546,0.005035,0.0,0.034576,0.018890,0.0,0.0,0.0,0.0
4,0.046204,0.253662,0.136353,0.0,0.0,0.032043,0.373047,0.245239,0.032715,0.178345,...,0.253662,0.178345,0.008408,0.0,0.046204,0.025223,0.0,0.0,0.0,0.0


In [13]:
train_scaled_nosplit.columns

Index(['time_step', 'u_in', 'time_delta', 'cross', 'area_out', 'area',
       'air_flow_rate', 'air_flow_area', 'u_in_cumsum', 'u_in_cummean',
       'time_step_cumsum', 'time_step_cummean', 'cross_cumsum',
       'cross_cummean', 'u_in_lag_diff_1', 'u_in_lag_inverse_diff_1',
       'time_step_lag_diff_1', 'time_step_lag_inverse_diff_1',
       'cross_lag_diff_1', 'cross_lag_inverse_diff_1', 'u_in_lag_diff_2',
       'u_in_lag_inverse_diff_2', 'time_step_lag_diff_2',
       'time_step_lag_inverse_diff_2', 'cross_lag_diff_2',
       'cross_lag_inverse_diff_2', 'u_in_lag_diff_3',
       'u_in_lag_inverse_diff_3', 'time_step_lag_diff_3',
       'time_step_lag_inverse_diff_3', 'cross_lag_diff_3',
       'cross_lag_inverse_diff_3', 'u_in_lag_diff_4',
       'u_in_lag_inverse_diff_4', 'time_step_lag_diff_4',
       'time_step_lag_inverse_diff_4', 'cross_lag_diff_4',
       'cross_lag_inverse_diff_4', '15_u_in_sum', '15_u_in_min', '15_u_in_max',
       '15_u_in_mean', '15_time_step_sum', '15_

In [14]:
text = "text123456.sav"
text[:-4]

'text123456'

In [15]:
#We will save the model performance metrics in a DataFrame
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
import pickle
import numpy as np
Model = []
RMSE = []
R_sq = []
cv = KFold(5, random_state = 1,shuffle=True)

#Creating a Function to append the cross validation scores of the algorithms
def input_scores(name, model, x, y):
    Model.append(name)
    print(name," - Started")
    rmse_tmp = np.sqrt((-1) * cross_val_score(model, x, y, cv=cv, scoring='neg_mean_squared_error').mean())
    RMSE.append(rmse_tmp)
    print(name, " - RSME - ", rmse_tmp)
    #Pickle Dump
    #file_name = f"{name}.sav"
    pickle.dump(model,open(f"{name}.h5","wb"))
    #pickle.dump(model,open(file_name,'wb'))
    #with open(f'{file_name[:-4]}','wb') as files:
        #pickle.dump(model,files)
    print(name," - Saved")
    
    R_sq_temp = cross_val_score(model, x, y, cv=cv, scoring='r2').mean()
    R_sq.append(R_sq_temp)
    print("R_sq - ", R_sq_temp)
    print("*-*"*20)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                              AdaBoostRegressor)

names = ['Linear Regression', 'Ridge Regression', 'Lasso Regression',
         'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor',
         'Adaboost Regressor']
models = [LinearRegression(), Ridge(), Lasso(),
          KNeighborsRegressor(), DecisionTreeRegressor(),
          RandomForestRegressor(), GradientBoostingRegressor(), 
          AdaBoostRegressor()]

#Running all algorithms
for name, model in zip(names, models):
    input_scores(name, model,X_train_nosplit, y_train_nosplit)

Linear Regression  - Started
Linear Regression  - RSME -  rmse_tmp
Linear Regression  - Saved
R_sq -  0.7368980399949455
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
Ridge Regression  - Started
Ridge Regression  - RSME -  rmse_tmp
Ridge Regression  - Saved


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 762, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 572, in fit
    X, y, X_offset, y_offset, X_scale = self._preprocess_data(
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_base.py", line 130, in _preprocess_data
    X = check_array(X, copy=copy, accept_sparse=['csr', 'csc'],
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\utils\validation.py

R_sq -  nan
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
Lasso Regression  - Started


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py", line 844, in fit
    self.path(X, y[:, k],
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py", line 530, in enet_path
    model = cd_fast.enet_coordinate_descent(
  File "sklearn\linear_model\_cd_fast.pyx", line 124, in sklearn.linear_model._cd_fast.enet_coordinate_descent
numpy.core._exceptions._ArrayMe

Lasso Regression  - RSME -  rmse_tmp
Lasso Regression  - Saved


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py", line 844, in fit
    self.path(X, y[:, k],
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py", line 530, in enet_path
    model = cd_fast.enet_coordinate_descent(
  File "sklearn\linear_model\_cd_fast.pyx", line 124, in sklearn.linear_model._cd_fast.enet_coordinate_descent
numpy.core._exceptions._ArrayMe

R_sq -  nan
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
K Neighbors Regressor  - Started


In [ ]:
evaluation = pd.DataFrame({'Model': Model,
                           'RMSE': RMSE,
                           'R Squared': R_sq})
print("FOLLOWING ARE THE TRAINING SCORES: ")
evaluation